## Data Visualization Project by Cyrus K Karuga

### Karamoja Subcounty; Food crop yield

#### step 1. Import all necessary Libraries

In [ ]:
###Import all libraries necessary to enable Transformation of data
import pandas as pd
from dbfread import DBF
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd


#### step 2. loading the data we have in our notebook

In [ ]:
# Load all the files that we need in our project
# lOADING DBF FILES
maize_dbf = pd.DataFrame(iter(DBF('Final_Project/Crop_Type_Map_Maize.dbf')))
sorghum_dbf = pd.DataFrame(iter(DBF('Final_Project/Crop_Type_Map_Sorghum.dbf')))
district_dbf = pd.DataFrame(iter(DBF('Final_Project/Uganda_Districts.dbf')))
subcounty_dbf = pd.DataFrame(iter(DBF('Final_Project/Uganda_Subcounties.dbf')))
# lOADING SHP FILES
maize_gdf = gpd.read_file('Final_Project/Crop_Type_Map_Maize.shp')
sorghum_gdf = gpd.read_file('Final_Project/Crop_Type_Map_Sorghum.shp')
district_gdf = gpd.read_file('Final_Project/Uganda_Districts.shp')
subcounty_gdf = gpd.read_file('Final_Project/Uganda_Subcounties.shp')

#Loading the data i.e .CSV file

df_district = pd.read_csv('Final_Project/Uganda_Karamoja_District_Crop_Yield_Population.csv')
df_subcounty = pd.read_csv('Final_Project/Uganda_Karamoja_Subcounty_Crop_Yield_Population.csv')

#### Step 2: Data Clean Up

#### Calculate:
1. Production per Capita for Sorghum and Maize
2. Yield Efficiency
3. Yield Efficiency for Sorghum
4. Yield Efficiency for Maize
5. Population Density
6. yield disparity
7. Land Utilization Efficiency
8. Comparative Advantage of crops

In [ ]:
# Calculate the percentage of land under sorghum and maize in the district and narrowed down to the subcounty,
# From the data provided, I reached to an assumption that the df_karamoja_District data is an extract from the df_karamoja_subcounty, a few errors..
#..pertaining the integrity of data were noted, hence I decided to use the former.
# lets call it m_pec, s_pec and o_pec for maize, sorghum and other crops respectively
def calculate_percentage(df, area_column, crop_area_column):
    return round((df[area_column] / df[crop_area_column]) * 100, 2)

df_district['m_pec%'] = calculate_percentage(df_district, 'M_Area_Ha', 'Crop_Area_Ha')
df_district['s_pec%'] = calculate_percentage(df_district, 'S_Area_Ha', 'Crop_Area_Ha')
df_district['o_pec%'] = round(((df_district['Crop_Area_Ha'] - (df_district['S_Area_Ha'] + df_district['M_Area_Ha'])) / df_district['Crop_Area_Ha']) * 100, 2)
    
df_subcounty['m_pec%'] = calculate_percentage(df_subcounty, 'M_Area_Ha', 'Crop_Area_Ha')
df_subcounty['s_pec%'] = calculate_percentage(df_subcounty, 'S_Area_Ha', 'Crop_Area_Ha')
df_subcounty['o_pec%'] = round(((df_subcounty['Crop_Area_Ha'] - (df_subcounty['S_Area_Ha'] + df_subcounty['M_Area_Ha'])) / df_subcounty['Crop_Area_Ha']) * 100, 2)


# Add a new column for Production per Capita for Sorghum
df_subcounty['S_Prod_Per_Capita'] = df_subcounty['S_Prod_Tot'] / df_subcounty['POP']

# Add a new column for Production per Capita for Maize
df_subcounty['M_Prod_Per_Capita'] = df_subcounty['M_Prod_Tot'] / df_subcounty['POP']

# Add a new column for Yield Efficiency for Sorghum
df_subcounty['S_Yield_Efficiency'] = df_subcounty['S_Prod_Tot'] / df_subcounty['S_Area_Ha']

# Add a new column for Yield Efficiency for Maize
df_subcounty['M_Yield_Efficiency'] = df_subcounty['M_Prod_Tot'] / df_subcounty['M_Area_Ha']

# Calculate Population Density 
df_subcounty['Population_Density'] = df_subcounty['POP'] / df_subcounty['Area']

# Calculate and  yield disparity (difference between highest and lowest yields)
sorghum_yield_disparity = df_subcounty['S_Yield_Ha'].max() - df_subcounty['S_Yield_Ha'].min()
maize_yield_disparity = df_subcounty['M_Yield_Ha'].max() - df_subcounty['M_Yield_Ha'].min()

# Land Utilization Efficiency
# Efficiency = Total Production / Crop Area
df_subcounty['Sorghum_Efficiency'] = df_subcounty['S_Prod_Tot'] / df_subcounty['S_Area_Ha']
df_subcounty['Maize_Efficiency'] = df_subcounty['M_Prod_Tot'] / df_subcounty['M_Area_Ha']


# Print the most and least efficient subcounties
most_efficient_sorghum = df_subcounty.iloc[df_subcounty['Sorghum_Efficiency'].idxmax()]['SUBCOUNTY_NAME']
least_efficient_sorghum = df_subcounty.iloc[df_subcounty['Sorghum_Efficiency'].idxmin()]['SUBCOUNTY_NAME']
most_efficient_maize = df_subcounty.iloc[df_subcounty['Maize_Efficiency'].idxmax()]['SUBCOUNTY_NAME']
least_efficient_maize = df_subcounty.iloc[df_subcounty['Maize_Efficiency'].idxmin()]['SUBCOUNTY_NAME']

# Calculate Comparative Advantage
df_subcounty['Sorghum_Advantage'] = df_subcounty['S_Prod_Tot'] > df_subcounty['M_Prod_Tot']
df_subcounty['Maize_Advantage'] = df_subcounty['M_Prod_Tot'] > df_subcounty['S_Prod_Tot']



#### Output

##### 1. Population Density

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_subcounty, x='Population_Density', y='S_Prod_Tot')
plt.title('Population Density vs Sorghum Production')
plt.xlabel('Population Density')
plt.ylabel('Total Sorghum Production')
plt.show()


# Scatter plot for Population Density vs Maize Production
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_subcounty, x='Population_Density', y='M_Prod_Tot')
plt.title('Population Density vs Maize Production')
plt.xlabel('Population Density')
plt.ylabel('Total Maize Production')
plt.show()


#### 2. Yield Disparities and Optimization Opportunities

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_subcounty[['S_Yield_Ha', 'M_Yield_Ha']], palette=['#3498db', '#2ecc71'])
sns.swarmplot(data=df_subcounty[['S_Yield_Ha', 'M_Yield_Ha']], color='black', alpha=0.6)
plt.title('Boxplot of Sorghum and Maize Yields per Hectare', fontsize=16)
plt.ylabel('Yield per Hectare', fontsize=14)
plt.xticks([0, 1], ['Sorghum Yield per Hectare', 'Maize Yield per Hectare'], fontsize=12)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

plt.show()


#### 3. Land Utilization Efficiency

In [ ]:
print(f'Most Efficient Subcounty for Sorghum: {most_efficient_sorghum}')
print(f'Least Efficient Subcounty for Sorghum: {least_efficient_sorghum}')
print(f'Most Efficient Subcounty for Maize: {most_efficient_maize}')
print(f'Least Efficient Subcounty for Maize: {least_efficient_maize}')

#### 4. Comparative Advantage and Crop Specialization

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(data=df_subcounty, x='SUBCOUNTY_NAME', y='S_Prod_Tot', color='blue', label='Sorghum Production')
sns.barplot(data=df_subcounty, x='SUBCOUNTY_NAME', y='M_Prod_Tot', color='green', label='Maize Production', alpha=0.7)
plt.title('Comparative Advantage in Sorghum and Maize Production')
plt.ylabel('Total Production')
plt.xticks(rotation=90)
plt.legend()
plt.show()

strong_sorghum_advantage = df_subcounty[df_subcounty['Sorghum_Advantage']]['SUBCOUNTY_NAME'].tolist()
strong_maize_advantage = df_subcounty[df_subcounty['Maize_Advantage']]['SUBCOUNTY_NAME'].tolist()

print(f'Subcounties with a Strong Sorghum Advantage: {strong_sorghum_advantage}')
print(f'Subcounties with a Strong Maize Advantage: {strong_maize_advantage}')


### Step 3. Export back to local machine


In [ ]:
df_subcounty.to_csv(r'C:\Users\cyrus.karuga\Downloads\DATA\DATA\f_subcounty.csv', index=False)